In [1]:
import pandas as pd
pd.options.display.max_columns = 99

In [2]:
first_five = pd.read_csv('loans_2007.csv', nrows=5)
first_five

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [4]:
first_five.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 52 columns):
id                            5 non-null int64
member_id                     5 non-null float64
loan_amnt                     5 non-null float64
funded_amnt                   5 non-null float64
funded_amnt_inv               5 non-null float64
term                          5 non-null object
int_rate                      5 non-null object
installment                   5 non-null float64
grade                         5 non-null object
sub_grade                     5 non-null object
emp_title                     3 non-null object
emp_length                    5 non-null object
home_ownership                5 non-null object
annual_inc                    5 non-null float64
verification_status           5 non-null object
issue_d                       5 non-null object
loan_status                   5 non-null object
pymnt_plan                    5 non-null object
purpose                       

In [8]:
ff_mb = first_five.memory_usage(deep=True).sum() / 2**20
ff_bytes = first_five.memory_usage(deep=True).sum()

In [9]:
print(ff_mb, ff_bytes)

0.00768661499023 8060


## Data Quality Issues from `first_five`

Based on looking at the `first_five` rows in `'loans_2007.csv'`, there are a few places for improvement with regard to data types:

- Convert to optimal float and object types (category)
    - All float values are float64, may be able to save some memory usage by downcasting to a lower float type.
    - A few object columns stand out for conversion to category types, namely:
        - `term`
        - `grade` and `subgrade`
        - `emp_length`
        - `home_ownership`
        - `verification_status`
        - `payment_status`
        - `payment_plan`
        - `purpose`
        - `addr_state`
        - `application_type`
    - Other's may also be eligible based on whether or not more less than 50% of values are unique.
- `zipcodes` are not full integers. The last two digits are blocked out with `xx`, presumably for privacy reasons. May not include this column and instead rely on the `addr_state` column for any geographic analysis.
- `revol_util` is an object because the column contains percentages in the format `XX.XX%`. It would be better to remove the `%` and convert the floats to decimals.
    - According to the data dictionary available at Lending Club's [website](https://www.lendingclub.com/info/download-data.action), `revol_util` refers to:  _Ratio of total current balance to high credit/credit limit for all revolving accounts_
  
Unfortunately, I cannot determine the the number of null values in the entire dataset from the first five rows. I'll attempt to calculate these values later in the analysis.

In [10]:
first_thousand = pd.read_csv('loans_2007.csv', nrows=1000)
total_memory = first_thousand.memory_usage(deep=True).sum() / 2**20

In [11]:
total_memory

1.5506629943847656

In [12]:
# based on mb usage of 1.55 MB at 1000 rows, where 5000 would put us over 
# 5MB, attempting the numbers 2500, 3000, 3500
first_2500 = pd.read_csv('loans_2007.csv', nrows=2500)
total_memory_2500 = first_2500.memory_usage(deep=True).sum() / 2**20
first_3000 = pd.read_csv('loans_2007.csv', nrows=3000)
total_memory_3000 = first_3000.memory_usage(deep=True).sum() / 2**20
first_3500 = pd.read_csv('loans_2007.csv', nrows=3500)
total_memory_3500 = first_3500.memory_usage(deep=True).sum() / 2**20

In [13]:
print(total_memory_2500)
print(total_memory_3000)
print(total_memory_3500)

3.8761548996
4.6512298584
5.42614555359


In [14]:
first_3250 = pd.read_csv('loans_2007.csv', nrows=3250)
total_memory_3250 = first_3250.memory_usage(deep=True).sum() / 2**20
total_memory_3250

5.0388936996459961

## Optimal Chunksize = 3000

Based on the DataQuest criteria for memory usage per batch (under 5MB/batch), working with 3000 rows at time will be the optimal chunksize for the `pd.read_csv()` method. 

For easy comparison, here are the total memory usage per number of rows to the hundreth decimal point:

- **1000 rows**: 1.55 MB
- **2500 rows**: 3.88 MB
- _**3000 rows**: 4.65 MB_
- **3250 rows**: 5.04 MB
- **3500 rows**: 5.43 MB

## Next Task: Stats/Chunk

Per DataQuest's steps, I am to find out for each chunk:

- How many columns have a numeric type? How many a string type? 
    - Based on information gleaned from the `first_five` dataframe, the answer to this question is
        - 30 columns are float64
        - 1 column is int64 (`id`)
        - 21 columns are objects (string-types)
- How many unique values are there in each string column? How many string columns contain values that are less than 50% unique?
- Which float columns have no missing values and could be candidates for conversion to integer type?
- Calculate total memory usage across all of the chunks.

In [16]:
# redoing the first_five.info() so stats mentioned in point 
# one is more easily accessible from this point
first_five.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 52 columns):
id                            5 non-null int64
member_id                     5 non-null float64
loan_amnt                     5 non-null float64
funded_amnt                   5 non-null float64
funded_amnt_inv               5 non-null float64
term                          5 non-null object
int_rate                      5 non-null object
installment                   5 non-null float64
grade                         5 non-null object
sub_grade                     5 non-null object
emp_title                     3 non-null object
emp_length                    5 non-null object
home_ownership                5 non-null object
annual_inc                    5 non-null float64
verification_status           5 non-null object
issue_d                       5 non-null object
loan_status                   5 non-null object
pymnt_plan                    5 non-null object
purpose                       

In [69]:
# running unique value count for string values
# groupby column
# len(Series.value_counts()) / len(chunk)

# null_counts on float columns
# groupy by column, isnull().sum()

# will run batching, concatenation in separate cell to help with
# debugging
data_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
unique_counts = {}

In [48]:
for chunk in data_iter:
    strings = chunk.select_dtypes(include=['object'])
    for col in strings.columns:
        if col in unique_counts:
            unique_counts[col].append(strings[col].value_counts())
        else:
            unique_counts[col] = [strings[col].value_counts()]

In [49]:
total_vcs = {}
for col in unique_counts:
    combined_vcs = pd.concat(unique_counts[col])
    final_values = combined_vcs.groupby(combined_vcs.index).sum()
    total_vcs[col] = final_values

In [50]:
total_vcs['term']

 36 months    31534
 60 months    11001
Name: term, dtype: int64

In [54]:
data_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
total_rows = 0
for chunk in data_iter:
    #print(len(chunk))
    total_rows += len(chunk)
print(total_rows)

42538


In [67]:
#determining which columns are less than 50%
lt_fifty = {}
for col in total_vcs:
    percentage = len(total_vcs[col]) / total_rows
    if percentage < 0.5:
        lt_fifty[col] = percentage

In [68]:
print(len(lt_fifty))

21


In [71]:
# it would appear almost all of the string columns are less than 50% 
# unique. the exception being 'emp_title'
data_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
null_counts = {}
for chunk in data_iter:
    floats = chunk.select_dtypes(include=['float'])
    for col in floats:
        if col in null_counts:
            null_counts[col].append(floats[col].isnull().sum())
        else:
            null_counts[col] = [floats[col].isnull().sum()]

In [75]:
total_null = {}
for col in null_counts:
    total_null[col] = sum(null_counts[col])

In [76]:
print(total_null)

{'total_pymnt': 3, 'revol_bal': 3, 'pub_rec': 32, 'last_pymnt_amnt': 3, 'inq_last_6mths': 32, 'total_rec_int': 3, 'total_rec_late_fee': 3, 'collection_recovery_fee': 3, 'total_pymnt_inv': 3, 'chargeoff_within_12_mths': 148, 'delinq_2yrs': 32, 'tax_liens': 108, 'delinq_amnt': 32, 'total_acc': 32, 'acc_now_delinq': 32, 'recoveries': 3, 'installment': 3, 'member_id': 3, 'pub_rec_bankruptcies': 1368, 'total_rec_prncp': 3, 'out_prncp_inv': 3, 'funded_amnt': 3, 'open_acc': 32, 'annual_inc': 7, 'policy_code': 3, 'funded_amnt_inv': 3, 'dti': 3, 'out_prncp': 3, 'loan_amnt': 3, 'collections_12_mths_ex_med': 148}


In [84]:
memory_usage = []
data_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
for chunk in data_iter:
    memory_usage.append(chunk.memory_usage(deep=True))

combined_mem = pd.concat(memory_usage)

In [85]:
total_memMB = combined_mem.sum() / 2**20
total_memMB

66.245146751403809

In [86]:
lt_fifty

{'addr_state': 0.0011754196248060557,
 'application_type': 2.3508392496121114e-05,
 'earliest_cr_line': 0.012459448022944191,
 'emp_length': 0.00028210070995345337,
 'grade': 0.0001645587474728478,
 'home_ownership': 0.00011754196248060558,
 'id': 0.08317269265127651,
 'initial_list_status': 2.3508392496121114e-05,
 'int_rate': 0.00926230664347172,
 'issue_d': 0.0012929615872866614,
 'last_credit_pull_d': 0.0025389063895810806,
 'last_pymnt_d': 0.002421364427100475,
 'loan_status': 0.00021157553246509004,
 'purpose': 0.0003291174949456956,
 'pymnt_plan': 4.701678499224223e-05,
 'revol_util': 0.02630589120315953,
 'sub_grade': 0.000822793737364239,
 'term': 4.701678499224223e-05,
 'title': 0.4999059664300155,
 'verification_status': 7.052517748836334e-05,
 'zip_code': 0.019676524519253374}

In [102]:
len(total_vcs['initial_list_status'])

1

## Observations

Based on observations of the data I plan to make the following changes to the data:

- Exclude the following columns:
    - `zipcode` as it ends in XX; will instead use `addr_state`
    - `application_type` and `initial_list_status` (only 1 value)
    - DataQuest recommend's removing `id` which refers to the loan id used by Lending Club. I'll follow this suggestion as it will not really be useful for analysis.
    - `pymnt_plan` as it only contains one instance of someone on a payment plan.
- Convert the following columns into categories:
    - `term` 
    - `grade`
    - `sub_grade`
    - `verification_status`
    - `home_ownership`
    - `loan_status`
    - `purpose`
    - `emp_length`
- Conver the following columns from objects to floats:
    - `int_rate`
    - `revol_util`
    - `term`
- Downcast floats to most appropriate type

In [2]:
ff = pd.read_csv('loans_2007.csv', nrows=5)
ff_floats = ff.select_dtypes(include=['float'])
ff_floats

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1296599.0,5000.0,5000.0,4975.0,162.87,24000.0,27.65,0.0,1.0,3.0,0.0,13648.0,9.0,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,171.62,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1314167.0,2500.0,2500.0,2500.0,59.83,30000.0,1.00,0.0,5.0,3.0,0.0,1687.0,4.0,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,119.66,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1313524.0,2400.0,2400.0,2400.0,84.33,12252.0,8.72,0.0,2.0,2.0,0.0,2956.0,10.0,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,649.91,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1277178.0,10000.0,10000.0,10000.0,339.31,49200.0,20.00,0.0,1.0,10.0,0.0,5598.0,37.0,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,357.48,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1311748.0,3000.0,3000.0,3000.0,67.79,80000.0,17.94,0.0,0.0,15.0,0.0,27783.0,38.0,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,67.79,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [13]:
data_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
null_count = []
for chunk in data_iter:
    chunk.dropna(how='all', inplace=True)
    floats = chunk.select_dtypes(include=['float'])
    null_count.append(floats.isnull().sum())
    

In [14]:
null_concat = pd.concat(null_count)
null_total = null_concat.groupby(null_concat.index).sum()

In [15]:
null_total

acc_now_delinq                  32
annual_inc                       7
chargeoff_within_12_mths       148
collection_recovery_fee          3
collections_12_mths_ex_med     148
delinq_2yrs                     32
delinq_amnt                     32
dti                              3
funded_amnt                      3
funded_amnt_inv                  3
inq_last_6mths                  32
installment                      3
last_pymnt_amnt                  3
loan_amnt                        3
member_id                        3
open_acc                        32
out_prncp                        3
out_prncp_inv                    3
policy_code                      3
pub_rec                         32
pub_rec_bankruptcies          1368
recoveries                       3
revol_bal                        3
tax_liens                      108
total_acc                       32
total_pymnt                      3
total_pymnt_inv                  3
total_rec_int                    3
total_rec_late_fee  

In [24]:
# there don't seem to be any numeric types that do not 
# contain null values - will downcast to floats for all
data_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
flt_rev_util = []
flt_count = 0
for chunk in data_iter:
    revol_util = chunk['revol_util']
    for value in revol_util:
        if type(value) == float:
            flt_count += 1
            flt_rev_util.append(value)


In [25]:
flt_count
# flt_rev_util is all nan values

93

In [27]:
cat_conv = {'term':'category', 'grade':'category', 'sub_grade':'category',
            'verification_status':'category', 'home_ownership': 'category', 
            'loan_status':'category', 'purpose':'category', 
            'emp_length':'category'}
drop_col = ['zip_code', 'application_type', 
            'initial_list_status', 'id', 'pymnt_plan']
tofloat = ['int_rate', 'revol_util', 'term']
data_iter = pd.read_csv('loans_2007.csv', dtype=cat_conv, chunksize=3000)
def rm_sign(string):
    try:
        return string.rstrip('%')
    except AttributeError:
        pass #meant to deal with nan values which raise AttributeErrors
def rm_months(string):
    return string.rstrip('months')
loans_data = []
memory = []
for chunk in data_iter:
    #downcast floats
    float_cols = chunk.select_dtypes(include=['float'])
    for col in float_cols:
        chunk[col] = pd.to_numeric(chunk[col], downcast='float')
    #remove %, months from columns in tofloat
    chunk['int_rate'] = chunk['int_rate'].apply(rm_sign)
    chunk['revol_util'] = chunk['revol_util'].apply(rm_sign)
    chunk['term'] = chunk['term'].apply(rm_months)
    for col in tofloat:
        chunk[col] = pd.to_numeric(chunk[col], downcast='float')
    #drop unwanted columns, inplace
    chunk.drop(labels=drop_col, axis=1, inplace=True)
    memory.append(chunk.memory_usage(deep=True))
    loans_data.append(chunk)

In [28]:
loans_data = pd.concat(loans_data)
loans_data.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1296599.0,5000.0,5000.0,4975.0,36.0,10.65,162.869995,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,credit_card,Computer,AZ,27.650000,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.699997,9.0,0.000000,0.000000,5863.155273,5833.839844,5000.000000,863.159973,0.000000,0.000000,0.00,Jan-2015,171.619995,Jun-2016,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1314167.0,2500.0,2500.0,2500.0,60.0,15.27,59.830002,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,car,bike,GA,1.000000,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.400000,4.0,0.000000,0.000000,1008.710022,1008.710022,456.459991,435.170013,0.000000,117.080002,1.11,Apr-2013,119.660004,Sep-2013,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1313524.0,2400.0,2400.0,2400.0,36.0,15.96,84.330002,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,small_business,real estate business,IL,8.720000,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.500000,10.0,0.000000,0.000000,3005.666748,3005.669922,2400.000000,605.669983,0.000000,0.000000,0.00,Jun-2014,649.909973,Jun-2016,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1277178.0,10000.0,10000.0,10000.0,36.0,13.49,339.309998,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,other,personel,CA,20.000000,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21.000000,37.0,0.000000,0.000000,12231.889648,12231.889648,10000.000000,2214.919922,16.969999,0.000000,0.00,Jan-2015,357.480011,Apr-2016,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1311748.0,3000.0,3000.0,3000.0,60.0,12.69,67.790001,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,other,Personal,OR,17.940001,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.900002,38.0,461.730011,461.730011,3581.120117,3581.120117,2538.270020,1042.849976,0.000000,0.000000,0.00,Jun-2016,67.790001,Jun-2016,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [31]:
memory = pd.concat(memory)
total_memory = sum(memory) / 2**20
total_memory

24.609990119934082

## Memory Savings

Huge savings from the original memory_usage by the lending club data. 

Originally, memory usage was **66.245146751403809MB**. After transforming the datatypes, memory usage was down to **24.609990119934082MB**.